Possible data sets:

https://www.kaggle.com/pontes/fake-news-sample

Real News: https://archive.ics.uci.edu/ml/datasets/News+Aggregator

https://toolbox.google.com/datasetsearch/search?query=fake%20news&docid=sHyIQgRMuTsFH02AAAAAAA%3D%3D

https://github.com/hanselowski/athene_system/tree/master/data

## Step 1 - Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import sklearn

#from sklearn.model_selection import train_test_split
#import sklearn.model_selection as ms
#import sklearn.feature_extraction.text as text
#import sklearn.naive_bayes as nb
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import BernoulliNB
#from sklearn.naive_bayes import GaussianNB
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score